This document is attributed to Alex Drlica-Wagner. For source file, click [here]("http://www-glast.stanford.edu/pub_data/1048/example3.py").

For more examples, data, and likefiles click [here]("http://www-glast.stanford.edu/pub_data/1048/").

In [1]:
import numpy as np
import pylab as plt
import matplotlib
%matplotlib inline

In [2]:
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy.optimize import brentq

In [3]:
def eflux(spectrum, emin=1e2, emax=1e5, quiet=False):
    # Integrate a generic spectrum, multiplied by E, to get the energy flux.
    espectrum = lambda e: spectrum(e)*e
    tol = min(espectrum(emin),espectrum(emax))*1e-10
    try:
        return quad(espectrum,emin,emax,epsabs=tol,full_output=True)[0]
    except (Exception, msg):
        print('Numerical error "%s" when calculating integral flux.' % msg)
        return np.nan

Keep numpy from complaining about dN/dE = 0...

In [4]:
np.seterr(divide='ignore',invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

Analyze Draco with 100 GeV b-bbar spectrum

In [5]:
likefile = 'http://www-glast.stanford.edu/pub_data/1048/like_draco.txt'
specfile = 'specfile2.txt'

J-Factor of Draco (*no J-factor uncertainty included*)

In [6]:
jfactor = 10**18.83

Spectral file created assuming J=10^18 GeV^2/cm^5, sigmav=1e-25 cm^3/s

In [7]:
j0 = 1e18
sigmav0 = 1e-25

Load the likelihood

In [8]:
data = np.loadtxt(likefile, unpack=True)
emins, emaxs = np.unique(data[0]),np.unique(data[1])
ebin = np.sqrt(emins*emaxs)
efluxes = data[2].reshape(len(emins),-1)
logLikes = data[3].reshape(len(emins),-1)

Load the spectrum. BE SURE TO CHECK UNITS The data is in GeV

In [9]:
energy,dnde = np.loadtxt(specfile,unpack=True)
energy *=1000
log_energy = np.log10(energy)
log_dnde = np.log10(dnde)
log_interp = interp1d(log_energy,log_dnde)
spectrum = lambda e: np.nan_to_num(10**( log_interp(np.log10(e)) ))

Predict energy flux from nominal spectral values


quick error fix: I let the interp1d function extrapolate. See the above 'log_interp'

In [10]:
pred = np.array([eflux(spectrum,e1,e2) for e1,e2 in zip(emins,emaxs)])

Interpolated log-likelihoods in each energy bin

In [11]:
likes = [ interp1d(f,l-l.max()) for f,l in zip(efluxes,logLikes) ]

Global log-likelihood summed over energy bins

In [12]:
like = lambda c: sum([lnlfn(c*p) for lnlfn,p in zip(likes,pred)])

Scan range for global log-likelihood

In [13]:
epsilon = 1e-4 #Just to make sure we stay within the interpolation range
xmin = epsilon
xmax = np.log10(efluxes.max()/efluxes[efluxes>0].min()) - epsilon
x = np.logspace(xmin,xmax,250)

norm0 = efluxes[efluxes>0].min() / pred.max()
norms = norm0 * x

Global log-likelihood using nominal value of nuisance parameter

In [14]:
lnl = np.array([like(n) for n in norms])

Convert global log-likelihood back into physical units

In [15]:
sigmav = j0/jfactor * sigmav0 * norms
lnlfn = interp1d(sigmav,lnl)
mle = lnl.max()
sigmav_mle = sigmav[lnl.argmax()]
delta = 2.71/2
limit = brentq(lambda x: lnlfn(x)-mle+delta, sigmav_mle, sigmav.max(), xtol=1e-10*sigmav_mle)
print("Upper Limit: %.5e" %limit)

Upper Limit: 3.31759e-43


In [16]:
energy

array([5.00000000e+02, 6.86298296e+02, 9.42010703e+02, 1.29300068e+03,
       1.77476833e+03, 2.43604096e+03, 3.34370152e+03, 4.58955332e+03,
       6.29960525e+03, 8.64681670e+03, 1.18685911e+04, 1.62907878e+04,
       2.23606798e+04, 3.06921929e+04, 4.21279994e+04, 5.78247484e+04,
       7.93700526e+04, 1.08943064e+05, 1.49534878e+05, 2.05251064e+05,
       2.81726911e+05, 3.86697399e+05, 5.30779532e+05, 7.28546177e+05])

In [17]:
emins

array([   500.      ,    666.760716,    889.139705,   1185.68685 ,
         1581.13883 ,   2108.48252 ,   2811.70663 ,   3749.47105 ,
         5000.      ,   6667.60716 ,   8891.39705 ,  11856.8685  ,
        15811.3883  ,  21084.8252  ,  28117.0663  ,  37494.7105  ,
        50000.      ,  66676.0716  ,  88913.9705  , 118568.685   ,
       158113.883   , 210848.252   , 281170.663   , 374947.105   ])

In [18]:
emaxs

array([   666.760716,    889.139705,   1185.68685 ,   1581.13883 ,
         2108.48252 ,   2811.70663 ,   3749.47105 ,   5000.      ,
         6667.60716 ,   8891.39705 ,  11856.8685  ,  15811.3883  ,
        21084.8252  ,  28117.0663  ,  37494.7105  ,  50000.      ,
        66676.0716  ,  88913.9705  , 118568.685   , 158113.883   ,
       210848.252   , 281170.663   , 374947.105   , 500000.      ])